# Cleanup

If you ran this Amazon Comprehend POC in your own account and would like to avoid ongoing charges, run the commands below to cleanup the resources created in the previous notebooks, including:

* S3 Bucket
* IAM Role
* Comprehend Jobs
* Comprehend classifier


## Pre-requisites

The cell below will import necessary libraries, retrieve stored variables from previous notebooks, and connect to Forecast, S3, and IAM via the Boto3 Python library.

In [ ]:
import sys
import os
import json
import time
from time import sleep
import boto3
import botocore.exceptions
import pandas as pd


# Retrieve stored variables
%store -r

# Create connections to Forecast, S3, and IAM
session = boto3.Session(region_name=region) 
comprehend = session.client(service_name='comprehend') 
s3 = session.client(service_name='s3')
iam = session.client(service_name='iam')


In [ ]:
def wait_till_delete(callback, check_time = 5, timeout = None):

    elapsed_time = 0
    while timeout is None or elapsed_time < timeout:
        try:
            out = callback()
        except botocore.exceptions.ClientError as e:
            # When given the resource not found exception, deletion has occured
            if e.response['Error']['Code'] == 'ResourceNotFoundException':
                print('Successful delete')
                return
            else:
                raise
        time.sleep(check_time)  # units of seconds
        elapsed_time += check_time

    raise TimeoutError( "Forecast resource deletion timed-out." )


## Delete Comprehend Custom Classifier


In [ ]:
# Delete comprehend custom classifier
wait_till_delete(lambda: comprehend.delete_document_classifier(DocumentClassifierArn=jobArn))


## Delete IAM and S3 Objects

Now that the Forecast objects are cleaned up, we can delete the rest of the IAM and S3 resources we created.

In [ ]:
# Detach Role Policies from the IAM Role
role_name = role_arn.split('/')[-1]
iamAttachedPolicyArns=[]
iamAttachedPolicyResponse=iam.list_attached_role_policies(RoleName=role_name)
for iap in iamAttachedPolicyResponse['AttachedPolicies']:
    iamAttachedPolicyArns.append(iap['PolicyArn'])
for ia in iamAttachedPolicyArns:
    iam.detach_role_policy(PolicyArn=ia, RoleName=role_name)

# Delete the IAM Role    
iam.delete_role(RoleName=role_name)


In [ ]:
# Delete all objects from the S3 Bucket
bucket = boto3.resource('s3').Bucket(bucket_name)
bucket.objects.all().delete()

# Delete the S3 Bucket
s3.delete_bucket(Bucket=bucket_name)

## Last Steps

Congratulations! You've successfully cleaned up the default lab objects from your account. If you created additional  resources that were not included in the default lab, be sure to clean those up manually or modify the scripts above to clean them up for you.

The final step is to go back to the CloudFormation console and delete the Stack you provisioned to create these notebooks.

We hope you enjoyed learning how to classify documents with Amazon Comprehend Custom Classifier.